In [ ]:
import os
import time
import numpy as np
from scipy.io import loadmat, savemat
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, coverage_error, label_ranking_average_precision_score, hamming_loss
from sklearn.multioutput import ClassifierChain

def main(scenario_name, snr_value, top_k=44):
    # Load data
    train_X = loadmat(f'/path/{scenario_name}-{snr_value}-train/data.mat')['datat']
    train_Y = loadmat(f'/path/{scenario_name}-{snr_value}-train-labels/labels.mat')['Labels']
    valid_X = loadmat(f'/path/{scenario_name}-{snr_value}-valid/data.mat')['datat']
    valid_Y = loadmat(f'/path/{scenario_name}-{snr_value}-valid-labels/labels.mat')['Labels']
    test_X = loadmat(f'/path/{scenario_name}-{snr_value}-test/data.mat')['datat']
    test_Y = loadmat(f'/path/{scenario_name}-{snr_value}-test-labels/labels.mat')['Labels']

    # Reshape data
    train_X = np.transpose(train_X, (3, 0, 1, 2)).reshape(6000, -1)
    valid_X = np.transpose(valid_X, (3, 0, 1, 2)).reshape(2000, -1)
    test_X = np.transpose(test_X, (3, 0, 1, 2)).reshape(2000, -1)

    # Concatenate train and validation data
    train_X = np.concatenate((train_X, valid_X))
    train_Y = np.concatenate((train_Y, valid_Y))

    # Standardize the data
    scaler = StandardScaler()
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.transform(test_X)

    # Reduce dimensionality with PCA
    pca = PCA(n_components=10)
    train_X = pca.fit_transform(train_X)
    test_X = pca.transform(test_X)

    # Initialize the base classifier with RBF kernel
    base_clf = SVC(kernel='rbf', probability=True, random_state=42)

    # Initialize the classifier chain with the base SVC estimator
    chain_classifier = ClassifierChain(base_estimator=base_clf, order='random', random_state=42)

    # Fit the classifier chain to the training data
    chain_classifier.fit(train_X, train_Y)
    start_inference_time = time.time()

    # Predict probabilities for the test set
    y_pred_prob =np.array(chain_classifier.predict_proba(test_X))
    print(y_pred_prob)
    sorted_indices = np.argsort(-y_pred_prob, axis=1)

    # Initialize array to store top k labels for each sample
    y_predSVM = np.zeros_like(test_Y)

    # Select top k labels for each sample
    for i in range(len(test_Y)):
        top_indices = sorted_indices[i, :top_k]
        y_predSVM[i, top_indices] = 1

    # Compute inference time
    
    end_inference_time = time.time()
    inference_time = end_inference_time - start_inference_time

    # Print the inference time
    print(f"Inference Time: {inference_time} seconds", flush=True)

    # Compute classification metrics
    report = classification_report(test_Y, y_predSVM)
    coverage_err = coverage_error(test_Y, y_pred_prob)
    avg_precision = label_ranking_average_precision_score(test_Y, y_pred_prob)
    hamming = hamming_loss(test_Y, y_predSVM)

    print("Classification Report:\n", report)
    print(f"Coverage Error: {coverage_err:.4f}")
    print(f"Label Ranking Average Precision Score: {avg_precision:.4f}")
    print(f"Hamming Loss: {hamming:.4f}")

    # Save predictions
    directory = "/your-path/"
    os.makedirs(directory, exist_ok=True)
    filename = f"{scenario_name}_{snr_value}_SVM_RBF_Chain.mat"
    filepath = os.path.join(directory, filename)

    try:
        savemat(filepath, {"y_predSVM": y_predSVM})
    except Exception as e:
        print("Error saving file:", e)

# Example usage:
if __name__ == "__main__":
    scenario_name = "44"
    snr_value = "snr5"
    main(scenario_name, snr_value)